# Sentiment Analysis with SBERT embeddings

## Our Dataset

This dataset describes the contents of the heart-disease diagnosis.

The dataset in this study is from [Kaggle](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data), which is called Twitter US Airline Sentiment.

- Dataset: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data

## Variable Table

| Original Dataset             | Data Type     | Description    |                                                         
|------------------------------|---------------|---------------------------------------------------------------------------------------------|
| tweet_id                     | ID            | A unique identifier for each tweet.                                                         | 
| airline_sentiment            | Categorical   | The sentiment expressed in the tweet (positive, neutral, negative).                         | 
| airline_sentiment_confidence | Numerical     | Confidence score in the sentiment label (0 to 1).                                           | 
| negativereason               | Categorical   | Reason for negative sentiment (e.g., "Late Flight", "Customer Service Issue").              | 
| negativereason_confidence    | Numerical     | Confidence score in the negative reason label (0 to 1).                                     | 
| airline                      | Categorical   | The airline mentioned in the tweet (e.g., United, Delta, etc.).                             | 
| airline_sentiment_gold       | Categorical   | Sentiment label by trusted annotator (gold standard).                                       | 
| name                         | Text          | Name of the user who posted the tweet.                                                      | 
| negativereason_gold          | Categorical   | Negative reason label by trusted annotator (gold standard).                                 | 
| retweet_count                | Numerical     | Number of times the tweet was retweeted.                                                    | 
| text                         | Text          | The full content of the tweet.                                                              | 
| tweet_coord                  | Geospatial    | Latitude and longitude coordinates where the tweet was posted, if available.                | 
| tweet_created                | Datetime      | Timestamp when the tweet was created.                                                       | 
| tweet_location               | Text          | Location specified in the user's profile.                                                   | 
| user_timezone                | Categorical   | Time zone specified in the user's profile.                                                  | 

<br/>

## Data Used for Modeling

| Feature                      | Data Type   | Description  |
|-----------------------------|-------------|--------------|
| **Target Variable: `encoded_sentiment`** | Categorical | This is an engineered variable derived from `airline_sentiment` for multi-class sentiment classification. It encodes sentiment as: 0 = Negative, 1 = Neutral, 2 = Positive. |
| **Feature: `text`**         | Text        | Contains consumer tweets about U.S. airlines. This field undergoes preprocessing, including removal of URLs and mentions (`@`), and stopword removal |


<br/>

# 1. Load Data

In [1]:
import pandas as pd
import warnings

# Load the dataset
df = pd.read_csv("../../data/tweets.csv.gz", compression="gzip")

warnings.filterwarnings("ignore")

In [2]:
# Show basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [3]:
# Show the first few rows
df.head(3)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)


# 2. Data Preprocessing

## 2.1 Handle Duplicates

In [4]:
# Check for duplicate rows
duplicate_rows = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Drop duplicate rows
df.drop_duplicates(inplace=True)

Number of duplicate rows: 36


In [5]:
# Confirm the shape after removal
print(f"Shape after dropping duplicates: {df.shape}")

Shape after dropping duplicates: (14604, 15)


## 2.2 Handle Missing Values

In [6]:
# Check for missing values for each variables in the dataset
print("\nMissing values count for each variables:")
print("-------------------------------------------")
print(df.isnull().sum())

print("""\n\n**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values""")


Missing values count for each variables:
-------------------------------------------
tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5445
negativereason_confidence        4101
airline                             0
airline_sentiment_gold          14564
name                                0
negativereason_gold             14572
retweet_count                       0
text                                0
tweet_coord                     13589
tweet_created                       0
tweet_location                   4723
user_timezone                    4814
dtype: int64


**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values


## 2.3 Text Processing

### 2.3.1 Feature Engineering

For feature engineering in sentiment analysis, we will perform the following steps:
- `Clean text`: lowercase the twitter text, and remove urls & mentions from the twitter text

In [7]:
import re

# Step 1: Lowercase and clean the text
def clean_text(text):
    text = text.lower()                                 # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text) # Remove URLs
    text = re.sub(r'@\w+', '', text)                    # Remove mentions
    # text = re.sub(r'#\w+', '', text)                    # NOTE: Do not remove hashtags, 
                                                                # as there is a lot of hashtags with sentiment indication, 
                                                                # such as '#thankyou', '#happycustomer', etc...
    # text = re.sub(r'[^a-z\s]', '', text)                # NOTE: no need to remove numbers and punctuation for sbert embedding
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Copy only the 'text' column to df_copy
df2 = df[['text', 'airline_sentiment']].copy()

# processed text after rule-based removing URL, twitter username
df2['clean_text'] = df2['text'].apply(clean_text)



In [8]:
df2.shape

(14604, 3)

In [9]:
# Check for missing values per column
print("\nMissing values per column:")
print(df2.isnull().sum())


Missing values per column:
text                 0
airline_sentiment    0
clean_text           0
dtype: int64


### 2.3.2 Target Engineering

We will convert the 'airline_sentiment' column into numerical values to use it as the target variable in our model, where `negative` = 0, `neutral` = 1, and `positive` = 2 

In [10]:
print("The target variable contains unique values of: ", df2['airline_sentiment'].unique(), 
      "which we are going to map it into 0, 1 and 2 respectively")

# Encode the sentiment column
df2['encoded_sentiment'] = df2['airline_sentiment'].map({"negative": 0, "neutral": 1, "positive": 2})

The target variable contains unique values of:  ['neutral' 'positive' 'negative'] which we are going to map it into 0, 1 and 2 respectively


## 3.0 Data Preparation for Modeling

In [11]:
df2

,text,airline_sentiment,clean_text,encoded_sentiment
0,@VirginAmerica What @dhepburn said.,neutral,what said.,1
1,@VirginAmerica plus you've added commercials t...,positive,plus you've added commercials to the experienc...,2
2,@VirginAmerica I didn't today... Must mean I n...,neutral,i didn't today... must mean i need to take ano...,1
3,@VirginAmerica it's really aggressive to blast...,negative,"it's really aggressive to blast obnoxious ""ent...",0
4,@VirginAmerica and it's a really big bad thing...,negative,and it's a really big bad thing about it,0
...,...,...,...,...
14635,@AmericanAir thank you we got on a different f...,positive,thank you we got on a different flight to chic...,2
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative,leaving over 20 minutes late flight. no warnin...,0
14637,@AmericanAir Please bring American Airlines to...,neutral,please bring american airlines to #blackberry10,1
14638,"@AmericanAir you have my money, you change my ...",negative,"you have my money, you change my flight, and d...",0


In [12]:
# Split into features (X) and target labels (y)
X = df2['clean_text']
y = df2['encoded_sentiment']

### 3.1 Train test validation split with stratified sampling

In [13]:
from sklearn.model_selection import train_test_split

# Train test validation split with ration 70:15:15
X_train, X_temp, y_train, y_temp = train_test_split(X, y, 
                                        stratify=y,            
                                        train_size=0.7, 
                                        random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, 
                                        stratify=y_temp,            
                                        train_size=0.5, 
                                        random_state=42)

In [14]:
# Export train and test dataset to `data` folder
pd.concat([X_train, y_train], axis=1).to_csv("../../data/train.csv.gz", index=False)
pd.concat([X_valid, y_valid], axis=1).to_csv("../../data/test.csv.gz", index=False)
pd.concat([X_test, y_test], axis=1).to_csv("../../data/test.csv.gz", index=False)


In [15]:
# Verify that class distribution is preserved after the train-test split (i.e., stratified correctly)

print("Train class distribution:")
print(y_train.value_counts(normalize=True))

print("\nValidation class distribution:")
print(y_test.value_counts(normalize=True))

print("\nTest class distribution:")
print(y_test.value_counts(normalize=True))

print("""\n**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.""")

Train class distribution:
encoded_sentiment
0    0.627177
1    0.211602
2    0.161221
Name: proportion, dtype: float64

Validation class distribution:
encoded_sentiment
0    0.627111
1    0.211775
2    0.161114
Name: proportion, dtype: float64

Test class distribution:
encoded_sentiment
0    0.627111
1    0.211775
2    0.161114
Name: proportion, dtype: float64

**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.


### 3.2 Text Representation with SBERT embedding

In [16]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer

EMBEDDING_TRAIN = "../../data/sbert_embedding_train.csv.gz"

if os.path.exists(EMBEDDING_TRAIN):
    X_train_sbert_embedding = pd.read_csv(EMBEDDING_TRAIN, compression="gzip", index_col=False)
else:
    # Load the SBERT model
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    X_train_sbert_embedding = sbert_model.encode(X_train.tolist(), show_progress_bar=True)

    # Save the sbert_embedding_train to `data` folder
    pd.DataFrame(X_train_sbert_embedding).to_csv(EMBEDDING_TRAIN, index=False)
    print(X_train_sbert_embedding)

In [17]:
import pandas as pd

EMBEDDING_VALID = "../../data/sbert_embedding_valid.csv.gz"
if os.path.exists(EMBEDDING_VALID):
    X_valid_sbert_embedding = pd.read_csv(EMBEDDING_VALID, compression="gzip", index_col=False)
else:
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    X_valid_sbert_embedding = sbert_model.encode(X_valid.tolist(), show_progress_bar=True)

    # Save the sbert_embedding_test to `data` folder
    pd.DataFrame(X_valid_sbert_embedding).to_csv(EMBEDDING_VALID, index=False)
    print(X_valid_sbert_embedding)

In [18]:
import pandas as pd

EMBEDDING_TEST = "../../data/sbert_embedding_test.csv.gz"
if os.path.exists(EMBEDDING_TEST):
    X_test_sbert_embedding = pd.read_csv(EMBEDDING_TEST, compression="gzip", index_col=False)
else:
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    X_test_sbert_embedding = sbert_model.encode(X_test.tolist(), show_progress_bar=True)

    # Save the sbert_embedding_test to `data` folder
    pd.DataFrame(X_test_sbert_embedding).to_csv(EMBEDDING_TEST, index=False)
    print(X_test_sbert_embedding)

In [19]:
X_train_vectorized = pd.read_csv("../../data/sbert_embedding_train.csv.gz", compression="gzip", index_col=False)
X_valid_vectorized = pd.read_csv("../../data/sbert_embedding_valid.csv.gz", compression="gzip", index_col=False)
X_test_vectorized = pd.read_csv("../../data/sbert_embedding_test.csv.gz", compression="gzip", index_col=False)

### 3.3 Handling class imbalance issue with SMOTE

In [20]:
from imblearn.over_sampling import SMOTE

# Handling imbalanced using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

In [21]:
# Check class distribution before and after applying SMOTE to confirm successful balancing

print("Class distribution in training set (before SMOTE):")
print(y_train.value_counts())

print("\nClass distribution in training set (after SMOTE):")
print(y_train_resampled.value_counts())

print("""\n**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.""")


Class distribution in training set (before SMOTE):
encoded_sentiment
0    6411
1    2163
2    1648
Name: count, dtype: int64

Class distribution in training set (after SMOTE):
encoded_sentiment
1    6411
0    6411
2    6411
Name: count, dtype: int64

**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.


## 4.0 Modeling

(i) Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
lr_model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and valid sets
y_train_pred_lr = lr_model.predict(X_train_resampled)
y_valid_pred_lr = lr_model.predict(X_valid_vectorized)
y_test_pred_lr = lr_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_lr)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_lr)
test_accuracy = accuracy_score(y_test, y_test_pred_lr)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_lr, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_lr, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_lr, average='weighted')
test_precision = precision_score(y_test, y_test_pred_lr, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_lr, average='weighted')
test_recall = recall_score(y_test, y_test_pred_lr, average='weighted')


# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Validation F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Validation Precision:  {valid_precision:.4f}")
print(f"Test Precision:  {test_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Validation Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_lr))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_lr))

Train Accuracy: 0.8352
Validation Accuracy:  0.7786
Test Accuracy:  0.7814

Train F1-score: 0.8352
Validation F1-score:  0.7875
Test F1-score:  0.7891

Train Precision: 0.8353
Validation Precision:  0.8098
Test Precision:  0.8076

Train Recall: 0.8352
Validation Recall:  0.7786
Test Recall:  0.7814

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.91      0.80      0.85      1374
           1       0.56      0.75      0.64       464
           2       0.73      0.77      0.75       353

    accuracy                           0.78      2191
   macro avg       0.73      0.77      0.75      2191
weighted avg       0.81      0.78      0.79      2191

Confusion Matrix (Test):
[[1093  220   61]
 [  77  347   40]
 [  30   51  272]]


(ii) Decision Tree

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the Decision Tree
dt_model = DecisionTreeClassifier(max_depth=100, 
                                  min_samples_split=10, 
                                  criterion='entropy', 
                                  min_samples_leaf=100,
                                  random_state=42)

# Train the model
dt_model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and test sets
y_train_pred_dt = dt_model.predict(X_train_resampled)
y_valid_pred_dt = dt_model.predict(X_valid_vectorized)
y_test_pred_dt = dt_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_dt)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_dt)
test_accuracy = accuracy_score(y_test, y_test_pred_dt)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_dt, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_dt, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_dt, average='weighted')
test_precision = precision_score(y_test, y_test_pred_dt, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_dt, average='weighted')
test_recall = recall_score(y_test, y_test_pred_dt, average='weighted')

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Valid Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Valid F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}")
print(f"Test Precision:  {test_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Valid Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_dt))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_dt))

Train Accuracy: 0.7100
Valid Accuracy:  0.6207
Test Accuracy:  0.6016

Train F1-score: 0.7097
Valid F1-score:  0.6382
Test F1-score:  0.6206

Train Precision: 0.7105
Valid Precision:  0.6850
Test Precision:  0.6750

Train Recall: 0.7100
Valid Recall:  0.6207
Test Recall:  0.6016

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.83      0.61      0.70      1374
           1       0.37      0.56      0.45       464
           2       0.46      0.63      0.53       353

    accuracy                           0.60      2191
   macro avg       0.55      0.60      0.56      2191
weighted avg       0.67      0.60      0.62      2191

Confusion Matrix (Test):
[[833 362 179]
 [118 261  85]
 [ 49  80 224]]


(iii) XGBoost

In [24]:
import os
import joblib
import xgboost as xgb
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the XGBoost Classifier
model_path = "../../models/sbert_embedding_xgb_model.pkl"
if os.path.exists(model_path):
    print("Loading existing model...")
    xgb_model = joblib.load(model_path)
else:
    # Train the model
    xgb_model = xgb.XGBClassifier(max_depth=10,
                                random_state=42,
                                # Introduce randomness to make training faster and reduce overfitting
                                subsample=0.8, ## Uses 80% of the data for each tree.
                                colsample_bytree=0.8, ## Uses 80% of the features for each tree.
                                # the parameters below make the model trained faster by enabling parallelism
                                n_jobs = -1)
    xgb_model.fit(X_train_resampled, y_train_resampled)
    # Export and load previously trained model to avoid retraining every time
    joblib.dump(xgb_model, model_path)
    print("Model saved.")

# Predictions on training and test sets
y_train_pred_xgb = xgb_model.predict(X_train_resampled)
y_valid_pred_xgb = xgb_model.predict(X_valid_vectorized)
y_test_pred_xgb = xgb_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_xgb)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_xgb)
test_accuracy = accuracy_score(y_test, y_test_pred_xgb)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_xgb, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_xgb, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_xgb, average='weighted')
test_precision = precision_score(y_test, y_test_pred_xgb, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_xgb, average='weighted')
test_recall = recall_score(y_test, y_test_pred_xgb, average='weighted')

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Valid Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Valid F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}")
print(f"Test Precision:  {test_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Valid Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_xgb))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_xgb))


Loading existing model...
Train Accuracy: 0.9986
Valid Accuracy:  0.8060
Test Accuracy:  0.8069

Train F1-score: 0.9986
Valid F1-score:  0.8054
Test F1-score:  0.8063

Train Precision: 0.9986
Valid Precision:  0.8060
Test Precision:  0.8062

Train Recall: 0.9986
Valid Recall:  0.8060
Test Recall:  0.8069

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      1374
           1       0.66      0.67      0.67       464
           2       0.74      0.67      0.70       353

    accuracy                           0.81      2191
   macro avg       0.76      0.74      0.75      2191
weighted avg       0.81      0.81      0.81      2191

Confusion Matrix (Test):
[[1218  110   46]
 [ 113  313   38]
 [  62   54  237]]
